In [1]:
import os
from dotenv import load_dotenv

MODEL = "llama3.1"
# MODEL = "gemma:2b"

In [2]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings()

# model.invoke("Tell me a joke")

In [3]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser
chain.invoke("Tell me a joke")

"Here's one:\n\nWhat do you call a fake noodle?\n\nAn impasta!\n\nHope that made you laugh! Do you want to hear another?"

In [4]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("iesc111.pdf")
pages = loader.load_and_split()
pages

[Document(metadata={'source': 'iesc111.pdf', 'page': 0}, page_content='Everyday we hear sounds from various\nsources like humans, bir ds, bells, machines,\nvehicles, televisions, radios etc. Sound is a\nform of ener gy which pr oduces a sensation\nof hearing in our ears. Ther e are also other\nforms of energy like mechanical energy, light\nenergy, etc. W e have talked about mechanical\nenergy in the pr evious chapters. Y ou have\nbeen taught about conservation of energy,\nwhich states that we can neither create nor\ndestr oy ener gy. W e  can just change it fr om\none for m to another . When you clap, a sound\nis produced. Can you produce sound without\nutilising your energy? Which form of energy\ndid you use to produce sound? In this\nchapter we are going to learn how sound is\nproduced and how it is transmitted through\na medium and received by our ears.\n11.1 Production of Sound\nActivity _____________ 11.1\n•Take a tuning fork and set it vibrating\nby striking its pr ong on a rubbe

In [5]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
print(prompt.format(context="Here is some context", question="Here is a question"))


Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: Here is some context

Question: Here is a question



In [6]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(
    pages, 
    embedding = embeddings
)

/Users/aniketsingh/Desktop/projects/NCERT AI Agent/rag_env/lib/python3.11/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [17]:
vector = embeddings.embed_query("How are you")
len(vector)

4096

In [7]:
retriever = vectorstore.as_retriever()
retriever.invoke("How is flaw detected using ultrasound?")

[Document(metadata={'source': 'iesc111.pdf', 'page': 1}, page_content='SCIENCE 128the vibrating object to the ear . A particle of\nthe medium in contact with the vibrating\nobject is first displaced fr om its equilibrium\nposition. It then exerts a for ce on the adjacent\nparticle. As a r esult of which the adjacent\nparticle gets displaced fr om its position of\nrest. After displacing the adjacent particle the\nfirst particle comes back to its original\nposition. This pr ocess continues in the\nmedium till the sound r eaches your ear . The\ndisturbance created by a source of  sound in\nthe medium travels through the medium and\nnot the particles of the medium.\nA wave is a disturbance that moves\nthrough a medium when the particles of the\nmedium set neighbouring particles into\nmotion. They in tur n produce similar motion\nin others. The particles of the medium do not\nmove forwar d themselves, but the\ndisturbance is carried forwar d. This is what\nhappens during pr opagation of sou

In [8]:
# from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

In [9]:
answer = chain.invoke({"question": "How does propagation of sound happens?"})
print(answer)

A particle of the medium in contact with the vibrating object is first displaced from its equilibrium position. It then exerts a force on the adjacent particle, which sets it vibrating. This process continues, and the vibrations are transferred through the medium, propagating the sound wave.


In [10]:
from pinecone.grpc import PineconeGRPC as Pinecone
load_dotenv()

pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

pc.list_indexes()

/Users/aniketsingh/Desktop/projects/NCERT AI Agent/rag_env/lib/python3.11/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


{'indexes': [{'deletion_protection': 'disabled',
              'dimension': 4096,
              'host': 'ncert-assistant-hoe9x7y.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'ncert-assistant',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}

In [15]:
from pinecone import ServerlessSpec

index_name = "ncert-assistant"
existing_indexes = pc.list_indexes()
print(existing_indexes)
if not pc.exists(index_name):
  pc.create_index(
    name=index_name,
    dimension=1536,
    metric='cosine',
    spec=ServerlessSpec(
      cloud="aws",
      region="us-east-1"
    )
  )

{'indexes': [{'deletion_protection': 'disabled',
              'dimension': 4096,
              'host': 'ncert-assistant-hoe9x7y.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'ncert-assistant',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}


AttributeError: 'PineconeGRPC' object has no attribute 'exists'